In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import PIL
import pathlib
import tensorflow as tf
import keras
from keras import models, layers
from sklearn.utils import compute_class_weight
from sklearn.metrics import classification_report
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
from keras import callbacks

#  Depending on the study, the associated DL model should be imported
from tensorflow.keras.applications import ResNet50


pretrained_model = ResNet50(input_shape = (224, 224, 3),
                               include_top = False,
                               weights = 'imagenet', 
                             pooling = 'max')

for layer in pretrained_model.layers:
    layer.trainable = False

 
#  This section defined the initialisation considerations for a dererministic model. 
#  If this section is commented, we will hava a stochastic model. 
# The initialisations are further described in:

# R. N. M. Rudd-Orthner and L. Mihaylova, "Non-Random Weight Initialisation in Deep Learning
# Networks for Repeatable Determinism," 2019 10th International Conference on Dependable Systems, 
# Services and Technologies (DESSERT), Leeds, UK, 2019, pp. 223-230, doi: 10.1109/DESSERT.2019.8770007.

keras.utils.set_random_seed(42) 
################## For Linear ramp initialisation - Scheme 2
#  It is worth noticing that the number of parameters of the model would affect the 
# constant values (e.g. 2048*128 && 4*128) of the initialisation. These should be defined before executing the code
initval1=np.arange(0,2048*128,1) /(2048*128-1)*0.1-0.05 
initval2=np.arange(0,4*128,1)/(4*128-1)*0.1-0.05

################## For Sinusoidal initialisation - Scheme 3
# initval1 = np.sin(np.arange(0,128*2048,1)/(128*2048-1)*np.pi*2)*0.05
# initval2 = np.sin(np.arange(0,4*128,1)/(4*128-1)*np.pi*2)*0.05    


#  model configuration
model = Sequential()
model.add(pretrained_model)
model.add (layers.Flatten())
model.add(layers.Dense(128, kernel_initializer = tf.constant_initializer(initval1),
                 bias_initializer = 'zeros' , activation = 'relu' ))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(4 , kernel_initializer = tf.constant_initializer(initval2),
                 bias_initializer = 'zeros' , activation = 'softmax'))

model.summary()


In [ ]:
#  The path to the dataset (the folder containing train and test set) for each specific colormap should be defined
DataPath = '../ColorData/Batlow/'

img_x , img_y = 224, 224
batch_size = 64


train = ImageDataGenerator(validation_split = 0.2)
test = ImageDataGenerator()


training1 = train.flow_from_directory(DataPath + 'training/' ,
                                          target_size=(img_x , img_y),
                                          batch_size = batch_size,
                                          class_mode = 'categorical',
                                      subset = 'training',
                                      seed = 123,
                                        shuffle = False)
validation1 = train.flow_from_directory(DataPath + 'training/',
                                          target_size=(img_x , img_y),
                                          batch_size = batch_size,
                                          class_mode = 'categorical',
                                        seed = 123,
                                      subset = 'validation',
                                        shuffle = False)                                       
testing1 = test.flow_from_directory( DataPath + 'testing/',
                                          target_size=(img_x , img_y),
                                          batch_size =batch_size,
                                          class_mode = 'categorical',
                                          shuffle = False)

In [ ]:
#  The early stopping and callback code should be executed for Schemes 2 to 4. and commented for Scheme 1. 

# earlystopping = callbacks.EarlyStopping(monitor="val_loss",
#                                         mode="min", patience= 5,
#                                         restore_best_weights=True,
#                                        verbose = 1)

model.compile(optimizer = Adam(learning_rate= 1e-4),
              loss = "categorical_crossentropy",
              metrics = ['accuracy'])
history1 = model.fit(training1, 
                     validation_data = validation1,
                     epochs = 100)
                    # callbacks = [earlystopping])

In [ ]:
model.evaluate(testing1, return_dict=True)

In [ ]:
import sys
#  Used for McNemar test
true_labels = testing1.classes
np.savetxt(sys.stdout, true_labels) 

In [ ]:
#  Used for McNemar test
predictions = model.predict(testing1)
pred_labels = list(np.argmax(predictions, axis=-1))
np.savetxt(sys.stdout, pred_labels)

In [ ]:
# get the confusion plot
from sklearn import metrics
cm = metrics.confusion_matrix(true_labels, pred_labels)
cm